In [16]:
from pymystem3 import Mystem
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import LatentDirichletAllocation
from transformers import BertTokenizer, BertModel
import torch
from transformers import pipeline
import spacy
from torch.utils.data import DataLoader, Dataset

In [19]:
import torch_directml
import torch

dml = torch_directml.device()
print(dml)  # Should return the DirectML device

privateuseone:0


In [17]:
import torch
print(torch.cuda.is_available())  # Should return True if ROCm is properly installed
print(torch.cuda.get_device_name(0))  # Should return the name of your AMD GPU


False


AssertionError: Torch not compiled with CUDA enabled

In [2]:
df_raw = pd.read_csv('./data/wildberries_reviews.csv.gz', compression='gzip')

In [3]:
df_raw.head()

,Unnamed: 0,review_full_text,review_rating,product,category,url
0,0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
1,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
2,2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
3,3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...
4,4,"Всё в комплекте, есть инструкция на русском яз...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...


In [4]:
df_test = df_raw.drop(["Unnamed: 0"], axis=1)

In [5]:
df_test.groupby("category").count().sort_values(by="review_full_text", ascending=False)

,review_full_text,review_rating,product,url
category,,,,
/Дом/Кухня/Кухонный текстиль/Рушники,2705,2705,2705,2705
/Для ремонта/Инструменты и оснастка/Монтажные инструменты,2586,2586,2586,2586
/Спорт/Командные виды спорта/Баскетбол/Форма,2574,2574,2574,2574
/Продукты/Бакалея/Макароны,2572,2572,2572,2572
/Дом/Хозяйственные товары/Товары для уборки,2559,2559,2559,2559
...,...,...,...,...
/Книги/Репринтные издания/Репринтные издания 1800 - 1809 годов,1,1,1,1
/Книги/Репринтные издания/Репринтные издания на английском языке авторов A-M,1,1,1,1
/Транспортные средства/Мототехника,1,1,1,1


In [6]:
df = df_test[df_test["category"] == "/Дом/Хозяйственные товары/Товары для уборки"]

In [7]:
# Предобработка данных
def preprocess_text(text):
    text = re.sub(r'<.*?>', '', text)  # Удаление HTML-тегов
    text = re.sub(r'[^а-яА-ЯёЁ\s]', '', text)  # Удаление лишних символов
    text = text.lower()  # Приведение к нижнему регистру
    return text

df['cleaned_text'] = df['review_full_text'].apply(preprocess_text)
df['cleaned_text']

C:\Users\Kirill\AppData\Local\Temp\ipykernel_23700\3431328801.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_text'] = df['review_full_text'].apply(preprocess_text)


989079          хорошие насадки швы ровные крепкие кармашки
989080    решила заказать тряпки эти  не увидела  что он...
989081    хорошие тряпки всё на месте моют отлично брала...
989082    отличные тряпки в комплекте  штуки очень хорош...
989083    отличные насадочки причем с обеих сторон нашив...
                                ...                        
991633    прикольная штучка попробовала пылюку убирает у...
991634                                  всё отлично спасибо
991635                          отличная щетка просто супер
991636    хорошая штука когда высокие потолки сделана  к...
991637                           классная качественная вещь
Name: cleaned_text, Length: 2559, dtype: object

In [8]:
# Извлечение признаков с помощью TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = vectorizer.fit_transform(df['cleaned_text'])

In [9]:
# Кластеризация с использованием K-means
kmeans = KMeans(n_clusters=10, random_state=42)
df['cluster'] = kmeans.fit_predict(X_tfidf)

C:\Users\Kirill\AppData\Local\Temp\ipykernel_23700\2781498300.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cluster'] = kmeans.fit_predict(X_tfidf)


In [10]:
# Тематическое моделирование с использованием LDA
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(X_tfidf)
df['topic'] = lda.transform(X_tfidf).argmax(axis=1)

C:\Users\Kirill\AppData\Local\Temp\ipykernel_23700\522459887.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['topic'] = lda.transform(X_tfidf).argmax(axis=1)


In [11]:
# Настройка устройства (CPU или GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [12]:
# Инициализация модели и токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased').to(device)

In [13]:
# Создание кастомного датасета
class ReviewsDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        inputs = self.tokenizer(text, return_tensors='pt', max_length=self.max_len, truncation=True, padding='max_length')
        return {key: val.squeeze(0) for key, val in inputs.items()}

In [14]:
# Создание DataLoader для пакетной обработки
dataset = ReviewsDataset(df['cleaned_text'].tolist(), tokenizer, max_len=512)
dataloader = DataLoader(dataset, batch_size=16, shuffle=False)

# Функция для извлечения эмбеддингов
def get_bert_embeddings(dataloader, model, device):
    model.eval()
    embeddings = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu().numpy()
            embeddings.extend(batch_embeddings)
    return embeddings

# Извлечение эмбеддингов
df['bert_embeddings'] = get_bert_embeddings(dataloader, model, device)

KeyboardInterrupt: 

In [ ]:
# Извлечение ключевых характеристик с использованием синтаксического анализа
nlp = spacy.load('ru_core_news_sm')

def extract_key_features(text):
    doc = nlp(text)
    features = []
    for token in doc:
        if token.pos_ == 'NOUN':
            feature = token.text
            for child in token.children:
                if child.pos_ in ['ADJ', 'VERB', 'NOUN']:
                    feature += ' ' + child.text
            features.append(feature)
    return features

df['key_features'] = df['cleaned_text'].apply(extract_key_features)

In [ ]:
# Анализ тональности
sentiment_analysis = pipeline('sentiment-analysis', model='blanchefort/rubert-base-cased-sentiment')
df['sentiment'] = df['cleaned_text'].apply(lambda x: sentiment_analysis(x)[0]['label'])

In [ ]:
# Сохранение результатов
df.to_csv('processed_reviews.csv', index=False)

print("Обработка завершена. Результаты сохранены в 'processed_reviews.csv'.")